In [1]:
import os
import pandas as pd
import pickle
import google.oauth2.credentials

import googleapiclient.discovery

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret.
CLIENT_SECRETS_FILE = "client_secret.json"

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'


def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [35]:
def main(url):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = "YOUR_API_KEY"
    service = get_authenticated_service()
    
# Get Video ID from URL
    splitURL=url.split("v=")
    videoId=splitURL[-1]

#Get video title
    requestVideo = service.videos().list(
        part="snippet,contentDetails,statistics",
        id=videoId
    )
    responseVideo = requestVideo.execute()
    
    for item in responseVideo['items']:
        videoTitle=item['snippet']['title']
        
# Get comments for the required video
    requestComment = service.commentThreads().list(
        part="snippet",
        videoId=videoId
    )
    responseComment = requestComment.execute()
    
    comment = []
    comment_id = []
    author=[]
    reply_count = []
    like_count = []
    
    i = 0    
    no_of_pages = 30
      
    while i < no_of_pages:
        i=i+1  
        print(i)
        # Check if another page exists
        if 'nextPageToken' in responseComment:
            requestComment = service.commentThreads().list(
                part="snippet",
                videoId=videoId,
                pageToken=responseComment['nextPageToken'])
            responseComment = requestComment.execute()
        # Add info to lists
            for item in responseComment['items']:
                comment.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
                comment_id.append(item['snippet']['topLevelComment']['id'])
                author.append(item['snippet']['topLevelComment']['snippet']['authorDisplayName'])
                reply_count.append(item['snippet']['totalReplyCount'])
                like_count.append(item['snippet']['topLevelComment']['snippet']['likeCount'])

    else:  
        doc=pd.DataFrame({ 
                      'Video Title':videoTitle,
                      'Comment': comment,
                      'Comment ID': comment_id,
                      'Author':author,
                      'Replies': reply_count,
                      'Likes': like_count})
      
    return(doc)      

        
if __name__ == "__main__":
    url = input('Enter an URL: ')
    doc=main(url)
 
    

Enter an URL: https://www.youtube.com/watch?v=uQ2Wa_sWBbc
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [36]:
doc.shape

(471, 6)

In [37]:
doc['Comment ID'].nunique()

471

In [30]:
doc.head()

,Video Title,Comment,Comment ID,Author,Replies,Likes
0,3 Ways to Make Green Chutney / How to Prepare ...,I love your chutney,Ugx2qj53N-Cvn6Eabl14AaABAg,Krishna manju,0,0
1,3 Ways to Make Green Chutney / How to Prepare ...,I saw an ant in chutney,UgxiJLnuv2pNp_Qnn2l4AaABAg,sehar bedi,0,0
2,3 Ways to Make Green Chutney / How to Prepare ...,I am follower for most of ur recepies...they ...,Ugw6XhZR7se-eXm8j714AaABAg,Sej2015 Mody,0,0
3,3 Ways to Make Green Chutney / How to Prepare ...,3rd chutney is the best recipe thanks,Ugyoj7aU8OtIdj1j4RN4AaABAg,kartik d,0,0
4,3 Ways to Make Green Chutney / How to Prepare ...,Keep it up,UgyUqP1zIHO4Rz__mXZ4AaABAg,Runi Ratna Priya Gudise,0,1


In [20]:
doc[doc['Comment ID']=='Ugw2Z2b6o4rjCb4nGVx4AaABAg']

,Video Title,Comment,Comment ID,Author,Replies,Likes
145,Reading minds through body language | Lynne Fr...,I am the listener and thinker at thesame time ...,Ugw2Z2b6o4rjCb4nGVx4AaABAg,Mia jewel,0,0
